# Décorateurs

In [18]:
def sandwich(garniture):
    def wrapper(*args, **kwargs):
        print("/TTTTTTTT\\")
        price = garniture(*args, **kwargs)
        print("\\________/")

        return price + 2

    return wrapper

In [21]:
@sandwich
def parisien():
    print("Jambon")
    print("Beurre")

    return 8

@sandwich
def kebab(salade_tomate_oignons:bool=True):
    if salade_tomate_oignons:
        print("salade")
        print("tomates")
        print("oignons")
    print("viande")
    return 5.5

parisien()
print(parisien())
kebab(salade_tomate_oignons=True)

/TTTTTTTT\
Jambon
Beurre
\________/
/TTTTTTTT\
Jambon
Beurre
\________/
10
/TTTTTTTT\
salade
tomates
oignons
viande
\________/


7.5

## Exercices
### Premier, timeit

In [39]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        value = func(*args, **kwargs)
        end = time.time()
        print(end - start, "secondes se sont écoulées")

        return value

    return wrapper

@time_it
def do_something():
    for x in range(1_000_000_0):
        y = x ** 2


In [27]:
do_something()

0.35279369354248047 secondes se sont écoulées


## Second exercice

In [28]:
def count_calls(func):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        func(*args, **kwargs)
        count += 1

    def get_count():
        return count

    inner.nb_counts = get_count

    return inner

In [29]:
@count_calls
def do_something2():
    print("done")

In [30]:
do_something2()
do_something2()
print(do_something2.nb_counts())
do_something2()
do_something2()
do_something2()
print(do_something2.nb_counts())

done
done
2
done
done
done
5


## Troisième exercice

In [35]:
from collections import OrderedDict

def cachable(func):

    cached_values = OrderedDict()

    def inner(value):
        if value not in cached_values:
            cached_values[value] = func(value)

        cached_values.move_to_end(value)

        if len(cached_values) > 4:
            cached_values.popitem(last=False)

        return cached_values[value]

    return inner

In [40]:
@time_it
@cachable
def long_call(value:int):
    time.sleep(2)
    return value**2


In [42]:
print(long_call(2))
print(long_call(2))
print(long_call(5))
print(long_call(8))
print(long_call(5))

2.1457672119140625e-06 secondes se sont écoulées
4
9.5367431640625e-07 secondes se sont écoulées
4
9.5367431640625e-07 secondes se sont écoulées
25
0.0 secondes se sont écoulées
64
9.5367431640625e-07 secondes se sont écoulées
25
wrapper


### Notifications

In [50]:
def register(level=2):
    def inner(notification):
        to_notify.append((level, notification))
        return notification
    return inner

to_notify = []

@register(level=1)
def notify1():
    print("notification from first")

def notify2():
    print("notification from second")

@register(level=2)
def notify3():
    print("notification from third")

def send_notifications(level):
    for notification_level, notification in to_notify:
        if notification_level >= level:
            notification()

In [51]:
send_notifications(1)

notification from first
notification from third
